# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [248]:
import pandas as pd
import requests

# 1) Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. 
#    Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas.
#    Há linhas duplicadas?
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há linhas duplicadas.

(27028, 69)


(27028, 69)

In [249]:
# Verificando as colunas e os dados para maior consciência situacional de abordagem.
sinasc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27028 entries, 0 to 27027
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ORIGEM        27028 non-null  int64  
 1   CODESTAB      26913 non-null  float64
 2   CODMUNNASC    27028 non-null  int64  
 3   LOCNASC       27028 non-null  int64  
 4   IDADEMAE      27028 non-null  int64  
 5   ESTCIVMAE     26711 non-null  float64
 6   ESCMAE        26716 non-null  object 
 7   CODOCUPMAE    24121 non-null  float64
 8   QTDFILVIVO    25455 non-null  float64
 9   QTDFILMORT    24930 non-null  float64
 10  CODMUNRES     27028 non-null  int64  
 11  GESTACAO      25796 non-null  object 
 12  GRAVIDEZ      26949 non-null  object 
 13  PARTO         26979 non-null  object 
 14  CONSULTAS     27028 non-null  int64  
 15  DTNASC        27028 non-null  object 
 16  HORANASC      27007 non-null  float64
 17  SEXO          27024 non-null  object 
 18  APGAR1        26932 non-nu

In [250]:
# 2) Conte o número de valores missing por variável.
lista_colunas = list(sinasc.columns)
for coluna in lista_colunas:
    resultado = sinasc[coluna].isnull().sum()
    print(f'Na Coluna: {coluna} o número de valores missing é: {resultado}.\n')

Na Coluna: ORIGEM o número de valores missing é: 0.

Na Coluna: CODESTAB o número de valores missing é: 115.

Na Coluna: CODMUNNASC o número de valores missing é: 0.

Na Coluna: LOCNASC o número de valores missing é: 0.

Na Coluna: IDADEMAE o número de valores missing é: 0.

Na Coluna: ESTCIVMAE o número de valores missing é: 317.

Na Coluna: ESCMAE o número de valores missing é: 312.

Na Coluna: CODOCUPMAE o número de valores missing é: 2907.

Na Coluna: QTDFILVIVO o número de valores missing é: 1573.

Na Coluna: QTDFILMORT o número de valores missing é: 2098.

Na Coluna: CODMUNRES o número de valores missing é: 0.

Na Coluna: GESTACAO o número de valores missing é: 1232.

Na Coluna: GRAVIDEZ o número de valores missing é: 79.

Na Coluna: PARTO o número de valores missing é: 49.

Na Coluna: CONSULTAS o número de valores missing é: 0.

Na Coluna: DTNASC o número de valores missing é: 0.

Na Coluna: HORANASC o número de valores missing é: 21.

Na Coluna: SEXO o número de valores missing

In [251]:
# 3) Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, 
#    muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. 
#    São elas:['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 

selecao = sinasc.loc[:,['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
selecao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27028 entries, 0 to 27027
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LOCNASC     27028 non-null  int64  
 1   IDADEMAE    27028 non-null  int64  
 2   ESTCIVMAE   26711 non-null  float64
 3   ESCMAE      26716 non-null  object 
 4   QTDFILVIVO  25455 non-null  float64
 5   GESTACAO    25796 non-null  object 
 6   GRAVIDEZ    26949 non-null  object 
 7   CONSULTAS   27028 non-null  int64  
 8   APGAR5      26925 non-null  float64
dtypes: float64(3), int64(3), object(3)
memory usage: 1.9+ MB


In [252]:
# 4) Apgar é uma nota que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas 
#    principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. 
#    Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. 
#    Para esta seleção, conte novamente o número de linhas e o número de missings.
lista_colunas = list(selecao.columns)
for coluna in lista_colunas:
    resultado = selecao[coluna].isnull().sum()
    print(f'Na Coluna: {coluna} o número de valores missing é: {resultado}.')

Na Coluna: LOCNASC o número de valores missing é: 0.
Na Coluna: IDADEMAE o número de valores missing é: 0.
Na Coluna: ESTCIVMAE o número de valores missing é: 317.
Na Coluna: ESCMAE o número de valores missing é: 312.
Na Coluna: QTDFILVIVO o número de valores missing é: 1573.
Na Coluna: GESTACAO o número de valores missing é: 1232.
Na Coluna: GRAVIDEZ o número de valores missing é: 79.
Na Coluna: CONSULTAS o número de valores missing é: 0.
Na Coluna: APGAR5 o número de valores missing é: 103.


In [253]:
# Criando DataFrame para realizar eliminação dados faltantes.
df = pd.DataFrame(selecao)

df.dropna(subset=['APGAR5'],inplace=True)
df

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [254]:
# Visualização dos dados ainda faltantes nas outras colunas.
lista_colunas = list(df.columns)
for coluna in lista_colunas:
    resultado = df[coluna].isnull().sum()
    print(f'Na Coluna: {coluna} o número de valores missing é: {resultado}.')

Na Coluna: LOCNASC o número de valores missing é: 0.
Na Coluna: IDADEMAE o número de valores missing é: 0.
Na Coluna: ESTCIVMAE o número de valores missing é: 315.
Na Coluna: ESCMAE o número de valores missing é: 310.
Na Coluna: QTDFILVIVO o número de valores missing é: 1566.
Na Coluna: GESTACAO o número de valores missing é: 1216.
Na Coluna: GRAVIDEZ o número de valores missing é: 76.
Na Coluna: CONSULTAS o número de valores missing é: 0.
Na Coluna: APGAR5 o número de valores missing é: 0.


In [255]:
# 5) observe que as variáveis ['ESTCIVMAE', 'CONSULTAS'] possuem o código 9, que significa ignorado. 
#    Vamos assumir que o não preenchido é o mesmo que o código 9.
cd1 = df['ESTCIVMAE'].isnull()
df[cd1] = 9

cd2 = df['CONSULTAS'].isnull()
df[cd2] = 9


In [256]:
# 6) Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero.
cd3 = df['QTDFILVIVO'].isnull()
df[cd3] = 0

In [257]:
# 7) Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' 
#    como no item anterior) e preencha. Justifique.

lista_colunas = list(df.columns)
for coluna in lista_colunas:
    resultado = df[coluna].isnull().sum()
    print(f'Na Coluna: {coluna} o número de valores missing é: {resultado}.')
    
# seguiremos do a substituição dos valores missing pelo número da documentação classificado como 'Ignorados'

Na Coluna: LOCNASC o número de valores missing é: 0.
Na Coluna: IDADEMAE o número de valores missing é: 0.
Na Coluna: ESTCIVMAE o número de valores missing é: 0.
Na Coluna: ESCMAE o número de valores missing é: 243.
Na Coluna: QTDFILVIVO o número de valores missing é: 0.
Na Coluna: GESTACAO o número de valores missing é: 1083.
Na Coluna: GRAVIDEZ o número de valores missing é: 64.
Na Coluna: CONSULTAS o número de valores missing é: 0.
Na Coluna: APGAR5 o número de valores missing é: 0.


In [258]:
# Correção dos dados 'NaN' substituídos por '9' = Ignorados
cd4 = df['ESCMAE'].isnull()
df[cd4] = 9

cd5 = df['GESTACAO'].isnull()
df[cd5] = 9

cd6 = df['GRAVIDEZ'].isnull()
df[cd6] = 9

In [259]:
# Verificação se ainda existe qualquer dado tipo 'NaN'

lista_colunas = list(df.columns)
for coluna in lista_colunas:
    resultado = df[coluna].isnull().sum()
    print(f'Na Coluna: {coluna} o número de valores missing é: {resultado}.')

Na Coluna: LOCNASC o número de valores missing é: 0.
Na Coluna: IDADEMAE o número de valores missing é: 0.
Na Coluna: ESTCIVMAE o número de valores missing é: 0.
Na Coluna: ESCMAE o número de valores missing é: 0.
Na Coluna: QTDFILVIVO o número de valores missing é: 0.
Na Coluna: GESTACAO o número de valores missing é: 0.
Na Coluna: GRAVIDEZ o número de valores missing é: 0.
Na Coluna: CONSULTAS o número de valores missing é: 0.
Na Coluna: APGAR5 o número de valores missing é: 0.


In [260]:
# 8) O Apgar possui uma classificação indicando se o bebê passou por asfixia:
#  Entre 8 e 10 está em uma faixa 'normal'.
#  Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
#  Entre 4 e 5 significa 'asfixia moderada'.
#  Entre 0 e 3 significa 'asfixia severa'.
#    Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.
df.loc[df['APGAR5'] <= 3, 'CLASSE_APGAR'] = 'asfixia severa'
df.loc[(df['APGAR5'] > 3) & (df['APGAR5'] <= 5), 'CLASSE_APGAR'] = 'asfixia moderada'
df.loc[(df['APGAR5'] > 5) & (df['APGAR5'] <= 7), 'CLASSE_APGAR'] = 'asfixia leve'
df.loc[df['APGAR5'] > 7, 'CLASSE_APGAR'] = 'normal'
df

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,CLASSE_APGAR
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal


In [261]:
df_classe = df['CLASSE_APGAR'].value_counts()
pd.DataFrame(df_classe)

,CLASSE_APGAR
normal,24983
asfixia severa,1601
asfixia leve,287
asfixia moderada,54


In [262]:
# 9) Renomeie as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um underscore entre as palávras. 
#    Dica: repare que se você não quiser criar um dataframe novo, você vai precisar usar a opção inplace = True.
df.columns = df.columns.str.lower().str.replace(' ', '_')
df


,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,classe_apgar
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal
